## Import libs

In [ ]:
from tensorflow.keras.models import Model, Sequential 
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.optimizers.schedules import ExponentialDecay 
import cv2 
from tensorflow.keras.models import model_from_json 
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import numpy as np # linear algebra
import pandas as pd

## Data

In [ ]:
batch_size = 64
image_height, image_width = 224, 224

In [ ]:
initial_epochs = 10
fine_tune_epochs = 40
total_epochs = fine_tune_epochs + initial_epochs

In [ ]:
train_generator = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/train-images/train',
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    image_size=(image_height, image_width),
    batch_size=batch_size,
    seed=42,
    validation_split=0.2,
    subset="training",
)
class_names = train_generator.class_names
train_generator = train_generator.repeat(total_epochs)

In [ ]:
train_size = 40038

In [ ]:
# посмотрим на данные для обучения
plt.figure(figsize=(10, 10))
for images, labels in train_generator.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i].numpy().argmax(axis=0)])
        plt.axis("off")

In [ ]:
validation_generator = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/train-images/train',
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    image_size=(image_height, image_width),
    batch_size=batch_size,
    seed=42,
    validation_split=0.2,
    subset="validation",
)
validation_generator = validation_generator.repeat(total_epochs)

In [ ]:
validation_size = 10009

In [ ]:
test_generator = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/test-images/test_kaggle',
    labels=None,
    shuffle=False,
    image_size=(image_height, image_width),
    batch_size=batch_size,
    seed=42,
)

In [ ]:
test_size = 5000

## Preprocess

### Configure the dataset for performance
Используем буферизованную предварительную выборку для загрузки изображений с диска без блокировки операций ввода-вывода.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_generator = train_generator.prefetch(buffer_size=AUTOTUNE)
validation_generator = validation_generator.prefetch(buffer_size=AUTOTUNE)
# test_generator = test_generator.prefetch(buffer_size=AUTOTUNE)

### Use data augmentation
Сгенерируем дополнительные данные, чтобы у модели было больше данных для обучения и она меньше переобучалась

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

## Создаем базовую модель из предобученной сети

In [ ]:
base_model = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(image_height, image_width, 3),
)

In [ ]:
# задаем входной слой
inputs = tf.keras.Input(shape=(image_height, image_width, 3))
x = data_augmentation(inputs)
# применяем требуемый препроцессинг для inception_v3
x = tf.keras.applications.inception_v3.preprocess_input(x)
# добавляем саму модель
# и замораживаем веса, чтобы не изменить их в процессе обучения
base_model.trainable = False
x = base_model(x, training=False)
# добавляем верхние слои, которые и будем обучать
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# и слой для предсказания класса
predictions = Dense(len(class_names), activation='softmax')(x)

# модель для обучения
model = Model(inputs=inputs, outputs=predictions)

initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=100000,  
                               decay_rate=0.96) 
  
optimizer = Adam(learning_rate=lr_schedule) 

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'])

In [ ]:
# посмотрим на итоговую архитектуру
model.summary()

## Обучаем добавленные слои

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    '/kaggle/working/ckpt/checkpoint.model.keras',
    monitor='val_loss',
    verbose=1, save_best_only=True)

min_delta=0
patience=4
# train the model on the new data for a few epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_size // batch_size,
    epochs=initial_epochs,
    validation_data=validation_generator,
    validation_steps=validation_size // batch_size,
    callbacks=[
        EarlyStopping(
            monitor='val_loss', min_delta=min_delta, patience=patience),
        checkpoint,
    ],
    verbose=1
)

## Промежуточная кривая обучения

In [ ]:
import os
os.listdir('/kaggle/working')


In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Categorical Accuracy')
plt.plot(val_acc, label='Validation Categorical Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Categorical Accuracy')
plt.ylim([0, 0.5])
plt.title('Training and Validation Categorical Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,3.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


## Fine tuning

In [ ]:
# посмотрим на исходную архитектуру чтобы понять, сколько слоев будем дообучать
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
# разморозим базовую модель, чтобы дообучить её верхние слои
base_model.trainable = True
# заморозим нижние слои модели
# таким образом мы будем дообучать 2 верхних inception блока
for layer in base_model.layers[:249]:
  layer.trainable = False

# но надо заморозить batchNormalization слои
for layer in base_model.layers:
    if 'batch_normalization' in layer.name:
        layer.trainable = False

In [ ]:
# снова соберем модель, чтобы изменения вступили в силу
# необходимо так же понизить learning rate, чтобы не переобучить модель
lr_schedule_2 = ExponentialDecay(0.00001, decay_steps=100000,  
                               decay_rate=0.96) 
  
optimizer_2 = Adam(learning_rate=lr_schedule_2) 
model.compile(optimizer=optimizer_2,
              loss='categorical_crossentropy',metrics=['categorical_accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_size // batch_size,
    epochs=total_epochs,
    initial_epoch=len(history.epoch),
    validation_data=validation_generator,
    validation_steps=validation_size // batch_size,
    callbacks=[
        checkpoint
    ],
    verbose=1
)

## Итоговая кривая обучения

In [ ]:
acc += history_fine.history['categorical_accuracy']
val_acc += history_fine.history['val_categorical_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Categorical Accuracy')
plt.plot(val_acc, label='Validation Categorical Accuracy')
plt.ylim([0.2, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Categorical Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 3.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model_json = model.to_json() 
with open("/kaggle/working/emotion_model.json", "w") as json_file: 
    json_file.write(model_json) 
  
# save trained model weight in .h5 file 
model.save_weights('/kaggle/working/emotion_model.weights.h5') 

## Получение предсказания для тестовых данных

In [ ]:
LABELS_AND_INDICES = (
    ('anger', 0),
    ('contempt', 1),
    ('disgust', 2),
    ('fear', 3),
    ('happy', 4),
    ('neutral', 5),
    ('sad', 6),
    ('surprise', 7),
    ('uncertain', 8),
)

LABELS_TO_INDICES = dict(LABELS_AND_INDICES)
INDICES_TO_LABELS = dict((y, x) for x, y in LABELS_AND_INDICES)

INDICES_TO_LABELS, LABELS_TO_INDICES

In [ ]:
def predictions_to_indices(predictions: np.array):
    return predictions.argmax(axis=1)

def indices_to_labels(predictions: np.array):
    to_indices = np.vectorize(lambda i: INDICES_TO_LABELS[i])
    return to_indices(predictions)

def predictions_to_labels(predictions: np.array):
    indices = predictions_to_indices(predictions)
    labels = indices_to_labels(indices)
    return labels

In [ ]:
# получаем предсказания для тестовых данных и декодируем их в эмоции
probs = model.predict(test_generator, verbose=1)
np.save('predictions.npy', probs)
probs = np.load('predictions.npy')
preds = predictions_to_labels(probs)
preds[:10]

In [ ]:
def get_filename_from_path(path_to_file: str):
    return path_to_file.split('/')[-1]

submission = pd.DataFrame({
    'image_path': test_generator.file_paths,
    'emotion': preds
})
submission['image_path'] = submission['image_path'].apply(get_filename_from_path)
submission.head()

In [ ]:
# посмотрим на предсказанные эмоции
plt.figure(figsize=(10, 10))
for batch in test_generator.take(1):
    preds = model.predict(batch).argmax(axis=1)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(batch[i].numpy().astype("uint8"))
        plt.title(class_names[preds[i]])
        plt.axis("off")

## Отправка данных в Kaggle

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)
submission.to_csv('submission.csv', index=False)